<a href="https://colab.research.google.com/github/sourav044/Python/blob/master/NLP_Gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Notebook is created to demostrated concepts related to NPL.** 

Exploring Gensim, Spacy, Keras implementing Seq2Seq and LSTM backend tensorflow.





Installing packages 

In [1]:
!pip install PyPDF2
!pip install wordsegment
!pip install pip install py-thesaurus

     |████████████████████████████████| 81kB 2.5MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp36-none-any.whl size=61085 sha256=fbf3e908997f3b17d75536afa9bcfc14d121c8027c78b4df1555e133d6905a21
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2
     |████████████████████████████████| 4.8MB 2.8MB/s 
ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install


Imporing all packages 

In [2]:
import nltk
import spacy
import PyPDF2
import keras
import tensorflow as tf
import gensim
import numpy as np
import re
import spacy
from nltk.corpus import stopwords
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.similarities import WmdSimilarity
from nltk.tokenize import sent_tokenize
from nltk.tokenize import sent_tokenize
from keras.preprocessing.text import Tokenizer 
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.similarities import WmdSimilarity
from collections import Counter
from nltk.corpus import wordnet
from gensim.corpora import Dictionary
from wordsegment import load, segment
from numpy import array
from keras.preprocessing.text import one_hot
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from prettytable import PrettyTable
from gensim.summarization.summarizer import summarize
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords
from collections import Counter
from spacy import displacy
from nltk.stem import PorterStemmer
import spacy.cli

Using TensorFlow backend.


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


Installing NLTK package

In [3]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords') 
nltk.download('vader_lexicon')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

Downloading and assinging laguage package : https://spacy.io/usage/models

In [4]:
lang = spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


### Convert Word to Vector 

In [5]:
model = Word2Vec(common_texts, size=20, min_count=1)  # train word-vectors
print(model)

Word2Vec(vocab=12, size=20, alpha=0.025)


In [6]:
index = WmdSimilarity(common_texts, model)
# Make query.
query = ['human', 'interface', 'computer']
sims = index[query]
print(common_texts)
print(sims)

[['human', 'interface', 'computer'], ['survey', 'user', 'computer', 'system', 'response', 'time'], ['eps', 'user', 'interface', 'system'], ['system', 'human', 'system', 'eps'], ['user', 'response', 'time'], ['trees'], ['graph', 'trees'], ['graph', 'minors', 'trees'], ['graph', 'minors', 'survey']]
[1.         0.47839252 0.50898521 0.5123328  0.43032193 0.41206602
 0.43028804 0.41284649 0.40399012]


### Defining all the method's

In [0]:
# All methods has been imported from : https://github.com/explosion/spaCy

def combine(sub_keys, keywords_splits, lb, mb, ub):
    whitespace = ' '
    while mb != ub:
        keywords_splits.append(whitespace.join(sub_keys[lb: mb]))
        keywords_splits.append(whitespace.join(sub_keys[mb: ub]))
        mb += 1
    del sub_keys[0]
    if len(sub_keys) > 2:
        combine(sub_keys, keywords_splits, 0, 1, len(sub_keys))


def keywords_splitter(keywords, keywords_splits):
    for key in keywords:
        sub_keys = key.split()
        if len(sub_keys) > 2:
            combine(sub_keys, keywords_splits, 0, 1, len(sub_keys))


def pre_query(question_query):
    keywords = question_query.get_features()
    keywords = [feat.lower() for feat in keywords]
    whitespace = ' '
    keywords_splits = whitespace.join(keywords).split()
    keywords_splitter(keywords, keywords_splits)
    keywords_splits = list(set(keywords_splits + keywords))
    return keywords_splits

#search the query over the dictionary which has been already created!
def query2vec(query, dictionary):
    print("Searching: {0}".format(query))
    corpus = dictionary.doc2bow(query)
    return corpus

# Convert doc to vector 
# It tokenize the word 
# It creates bag of words 
def doc2vec(texts):    
    frequency = Counter()
    for sent in texts:
        for token in sent:
            frequency[token] += 1    
    text_token = [[token for token in snipp]for snipp in texts]
    texts = [[token for token in snipp if frequency[token] > -1]for snipp in texts]
    dictionary = gensim.corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(snipp) for snipp in texts]
    return corpus, dictionary, texts, text_token

# Using Gensim LSI Model to find best match 
def transform_vec(corpus, query_corpus):
    lsidf = gensim.models.LsiModel(corpus)
    corpus_lsidf = lsidf[corpus]
    query_lsidf = lsidf[query_corpus]
    return corpus_lsidf, query_lsidf


def similarity(corpus_lsidf, query_lsidf):
    index = gensim.similarities.SparseMatrixSimilarity(corpus_lsidf, num_features=200000)
    simi = index[query_lsidf]
    simi_sorted = sorted(enumerate(simi), key=lambda item: -item[1])
    return simi_sorted




Features method

In [0]:
def get_detail(sentence):
    for token in sentence:
        print("{0} -- Lemma:{1}, Tag:{2}, EntType:{3}, Dep:{4}, Head:{5}"
                     .format(token.text, token.lemma_, token.tag_, token.ent_type_, token.dep_, token.head))


def get_compound_nouns(en_doc, token, token_text):

    """
    Recursively find the left and right compound nouns
    """

    parent_token = token

    print("Compound Noun:{0} DEP {1}".format(token.text, token.dep_))

    # If previous token is a compound noun
    while token.i > 0 and en_doc[token.i - 1].dep_ == "compound":
        token_text = en_doc[token.i - 1].text + " " + token_text
        token = en_doc[token.i - 1]
        # if the compound noun has any adjective modifier
        token_text = get_adj_phrase(token, token_text)

    token = parent_token

    # If next token is a compound noun
    while token.i < len(en_doc) - 1 and en_doc[token.i + 1].dep_ == "compound":
        token_text = token_text + " " + en_doc[token.i + 1].text
        token = en_doc[token.i + 1]
        # if the compound noun has any adjective modifier
        token_text = get_adj_phrase(token, token_text)

    # NOTE: Can token.shape_ == Xxxx... or XXXX... token.ent_iob_ help us here ...?

    return token_text

def this_is_adjective(dep_):
    return dep_ == "amod" or dep_ == "acomp" or dep_ == "ccomp"

def get_adj_phrase(token, token_text):

    """
    To fetch all the adjectives describing the noun
    """

    # amod: An adjectival modifier of a noun is any adjectival phrase that serves to modify the meaning of the noun.
    # ccomp: A clausal complement of a verb or adjective is a dependent clause which is a core argument.
    #        That is, it functions like an object of the verb, or adjective.
    # acomp: An adjectival complement of a verb is an adjectival phrase which functions as the complement

    for child in token.children:
        if(this_is_adjective(child.dep_)) and child.text != "much" and child.text != "many":
            # if child.dep_ == "amod" or child.dep_ == "acomp" or child.dep_ == "ccomp":  # not for how many
            #     if child.text != "much" and child.text != "many":
            token_text = child.lemma_ + " " + token_text
    return token_text


def get_root_phrase(token, keywords):

    # xcomp: An open clausal complement (xcomp) of a verb or an adjective is a predicative or clausal complement
    #        without its own subject.

    for child in token.children:
        if child.dep_ == "acomp" or child.dep_ == "xcomp" or child.dep_ == "ccomp":
            keywords.append(child.lemma_)
    return keywords


def this_is_noun(tag_):
    # If is Noun/Proper Noun, be it Singular or Plural
    return tag_ == "NN" or tag_ == "NNP" or tag_ == "NNPS" or tag_ == "NNS"

# Finding the word tag and dep
def get_noun_chunk(sentence, en_doc, keywords):

    root_word = ""

    for token in sentence:

        # If the Noun itself is not a compound Noun then we can find its compound Nouns
        print("Token -" +str(token))
        if(this_is_noun(token.tag_)) and token.dep_ != "compound":
            token_text = get_compound_nouns(en_doc, token, token.text)
            keywords.append(token_text)

        if token.tag_ == "JJ" and token.dep_ == "attr":
            token_text = get_compound_nouns(en_doc, token, token.text)
            token_text = get_adj_phrase(token, token_text)
            keywords.append(token_text)

        # If is a Cardinal Number & dependency is numeric modifier
        # nummod : A numeric modifier of a noun is any number phrase that
        #          serves to modify the meaning of the noun with a quantity.
        if token.dep_ == "nummod" or token.tag_ == "CD":
            token_text = token.text

            if token.i > 0 and en_doc[token.i - 1].tag_ == "JJ":
                # If previous token is Adjective, the adjective is liked with the cardinal number
                token_text = en_doc[token.i - 1].text + " " + token.text

            if token.i < len(en_doc) - 1 and en_doc[token.i + 1].tag_ == "JJ":
                # If next token is Adjective
                token_text = token.text + " " + en_doc[token.i + 1].text

            keywords.append(token_text)

        # Extracts the root word of sentence
        if token.dep_ == "ROOT":
            root_word = token.lemma_
            print("Root Word -> "+ str(token))            
            keywords = get_root_phrase(token, keywords)

    return root_word, keywords

#
def extract_features(question_type, en_doc, show_debug=False):

    # NOTE: In the whole pipeline question_type is not used anywhere currently...

    keywords = []
    for sentence in en_doc.sents:      
      if show_debug:
        get_detail(sentence)
      root, keywords = get_noun_chunk(sentence, en_doc, keywords)
      keywords.append(root)

    return keywords

In [0]:
# output list 
def lsToString(s):  
    
    # initialize an empty string 
    str1 = ""  
    
    # traverse in the string   
    for ele in s: 
      if type(ele) == list:
        str1 += " " +lsToString(ele)
      else: 
        str1 += " " +ele   
    
    # return string   
    return str1.strip() 
  
# function used for removing nested  
# lists in python.  
def removNestings(l): 
    output = [] 
    for i in l: 
        if type(i) == list: 
           output.append(removNestings(i)) 
        else: 
            output.append(i) 
    return output
  

def keywordMaker(l): 
    result = []
    for i in l:    
      result.append(lsToString(removNestings(i)))      
    return result
  



### Exploring Lemmatization, Stemming, and BOG

*The goal of both stemming and lemmatization is to reduce inflectional forms and sometimes derivationally related forms of a word to a common base form.*   

Rather than using a stemmer, you can use a lemmatizer , a tool from Natural Language Processing which does full morphological analysis to accurately identify the lemma for each word.

Stemming: cats -> "cat"

In [10]:
porter = PorterStemmer()
print(porter.stem("cats"))

cat


Lemmatization : am, is, are -> “be”



In [39]:
input_text = "Harry Potter is a series of fantasy novels written by British author J. K. Rowling. The novels chronicle the lives of a young wizard, Harry Potter, and his friends Hermione Granger and Ron Weasley, all of whom are students at Hogwarts School of Witchcraft and Wizardry. The main story arc concerns Harry's struggle against Lord Voldemort, a dark wizard who intends to become immortal, overthrow the wizard governing body known as the Ministry of Magic and subjugate all wizards and Muggles (non-magical people). Since the release of the first novel, Harry Potter and the Philosopher's Stone, on 26 June 1997, the books have found immense popularity, critical acclaim and commercial success worldwide. They have attracted a wide adult audience as well as younger readers and are often considered cornerstones of modern young adult literature. As of February 2018, the books have sold more than 500 million copies worldwide, making them the best-selling book series in history, and have been translated into eighty languages.[3] The last four books consecutively set records as the fastest-selling books in history, with the final instalment selling roughly eleven million copies in the United States within twenty-four hours of its release."
en_doc = nlp(u'' + input_text)
sentences = list(en_doc.sents)
print(sentences)

[Harry Potter is a series of fantasy novels written by British author J. K. Rowling., The novels chronicle the lives of a young wizard, Harry Potter, and his friends Hermione Granger and Ron Weasley, all of whom are students at Hogwarts School of Witchcraft and Wizardry., The main story arc concerns Harry's struggle against Lord Voldemort, a dark wizard who intends to become immortal, overthrow the wizard governing body known as the Ministry of Magic and subjugate all wizards and Muggles (non-magical people)., Since the release of the first novel, Harry Potter and the Philosopher's Stone, on 26 June 1997, the books have found immense popularity, critical acclaim and commercial success worldwide., They have attracted a wide adult audience as well as younger readers and are often considered cornerstones of modern young adult literature., As of February 2018, the books have sold more than 500 million copies worldwide, making them the best-selling book series in history, and have been tran

In [12]:
texts = [[word for word in doc.lemma_.split() if word not in [stopwords]]for doc in sentences]

corpus, dictionary, text, token = doc2vec(texts)

print(texts) 
print(token)


[['Harry', 'Potter', 'be', 'a', 'series', 'of', 'fantasy', 'novel', 'write', 'by', 'british', 'author', 'J.', 'K.', 'Rowling', '.'], ['the', 'novel', 'chronicle', 'the', 'life', 'of', 'a', 'young', 'wizard', ',', 'Harry', 'Potter', ',', 'and', '-PRON-', 'friend', 'Hermione', 'Granger', 'and', 'Ron', 'Weasley', ',', 'all', 'of', 'whom', 'be', 'student', 'at', 'Hogwarts', 'School', 'of', 'Witchcraft', 'and', 'Wizardry', '.'], ['the', 'main', 'story', 'arc', 'concern', 'Harry', "'s", 'struggle', 'against', 'Lord', 'Voldemort', ',', 'a', 'dark', 'wizard', 'who', 'intend', 'to', 'become', 'immortal', ',', 'overthrow', 'the', 'wizard', 'governing', 'body', 'know', 'as', 'the', 'Ministry', 'of', 'Magic', 'and', 'subjugate', 'all', 'wizard', 'and', 'Muggles', '(', 'non', '-', 'magical', 'people', ')', '.'], ['since', 'the', 'release', 'of', 'the', 'first', 'novel', ',', 'Harry', 'Potter', 'and', 'the', 'Philosopher', "'s", 'Stone', ',', 'on', '26', 'June', '1997', ',', 'the', 'book', 'have', '

In [13]:
# Checking Dictionary, It will arrange as per no. of occurance. 
for i in range(10):
  print(dictionary[i])

.
Harry
J.
K.
Potter
Rowling
a
author
be
british


In [14]:
print(corpus) # Corpus : "A corpus is a large body of natural language text used for accumulating statistics on natural language text. 
# (word Index id, Count) 
# BOW

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)], [(0, 1), (1, 1), (4, 1), (6, 1), (8, 1), (12, 1), (13, 3), (16, 3), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 3), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 2), (34, 1), (35, 1), (36, 1)], [(0, 1), (1, 1), (6, 1), (13, 1), (16, 2), (26, 1), (27, 2), (33, 3), (35, 3), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1)], [(0, 1), (1, 1), (4, 1), (12, 1), (13, 1), (16, 4), (27, 2), (33, 4), (37, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1)], [(0, 1), (6, 

In [15]:
from gensim.corpora import Dictionary
dct = Dictionary(["máma mele maso".split(), "ema má máma".split()])
print(dct)
print("Bag of Words : - list of (token_id, token_count) tuples.")
print(dct.doc2bow(["this", "is", "máma"], return_missing=True))

Dictionary(5 unique tokens: ['maso', 'mele', 'máma', 'ema', 'má'])
Bag of Words : - list of (token_id, token_count) tuples.
([(2, 1)], {'is': 1, 'this': 1})


### **Question Answer**

In [16]:
question = "Who is Harry Potter?"
en_doc_l = nlp(u'' + question)
features = extract_features("", en_doc_l, True)
print("Extracted: {0}".format(features))

Who -- Lemma:who, Tag:WP, EntType:, Dep:attr, Head:is
is -- Lemma:be, Tag:VBZ, EntType:, Dep:ROOT, Head:is
Harry -- Lemma:Harry, Tag:NNP, EntType:PERSON, Dep:compound, Head:Potter
Potter -- Lemma:Potter, Tag:NNP, EntType:PERSON, Dep:attr, Head:is
? -- Lemma:?, Tag:., EntType:, Dep:punct, Head:is
Token -Who
Token -is
Root Word -> is
Token -Harry
Token -Potter
Compound Noun:Potter DEP attr
Token -?
Extracted: ['Harry Potter', 'be']


In [17]:
doc = nlp(question)
print([(X.text, X.label_) for X in doc.ents])

[('Harry Potter', 'PERSON')]


In [18]:

ques_feature = [word.split() for word in features]

mix_ques = ([[x,y] for x in ques_feature for y in ques_feature if x != y])

#listToString = str(','.join([str(elem) for elem in mix_ques]) + ","+ question +","+ ','.join([str(elem) for elem in ques])) 
 
print([' '.join([str(c) for c in lst]) for lst in mix_ques])


["['Harry', 'Potter'] ['be']", "['be'] ['Harry', 'Potter']"]


In [0]:
list_question = mix_ques
ques_make = keywordMaker(list_question)
ques_make.append(question)
search = ques_make + keywordMaker(ques_feature)

In [20]:
query_corpus = query2vec(search, dictionary)

Searching: ['Harry Potter be', 'be Harry Potter', 'Who is Harry Potter?', 'Harry Potter', 'be']


In [0]:
corpus_lsidf, query_lsidf = transform_vec(corpus, query_corpus)

In [22]:
print(corpus_lsidf)

In [0]:
simi_sorted = similarity(corpus_lsidf, query_lsidf)

In [24]:
print(len(simi_sorted))

7


In [25]:
    
    print("simi_sorted \n")
    print(simi_sorted)
    candidate_ans = []
    
    ##New Correction
    for sent in simi_sorted:
        sent_id = sent[0]
        if sent[1] > 0:
          candidate_ans.append(str(sentences[sent_id]))
          print(candidate_ans)
        
        
        

simi_sorted 

[(0, 0.7424258), (4, 0.5514601), (5, 0.42424336), (1, 0.4004348), (6, 3.958121e-09), (3, -1.8626451e-08), (2, -2.8871e-08)]
['Harry Potter is a series of fantasy novels written by British author J. K. Rowling.']
['Harry Potter is a series of fantasy novels written by British author J. K. Rowling.', 'They have attracted a wide adult audience as well as younger readers and are often considered cornerstones of modern young adult literature.']
['Harry Potter is a series of fantasy novels written by British author J. K. Rowling.', 'They have attracted a wide adult audience as well as younger readers and are often considered cornerstones of modern young adult literature.', 'As of February 2018, the books have sold more than 500 million copies worldwide, making them the best-selling book series in history, and have been translated into eighty languages.[3]']
['Harry Potter is a series of fantasy novels written by British author J. K. Rowling.', 'They have attracted a wide adult 

In [26]:

matrix_tag = []
w1 = wordnet.synset('date.n.01') 
w2 = wordnet.synset('time.n.01') # n denotes noun 
print(w1.wup_similarity(w2)) 
for statement in candidate_ans:
  selected_statement =  nlp(statement)
  for ent in selected_statement.ents:   
    matrix_tag += [ent.text,ent.label_+"("+spacy.explain(ent.label_)+")"]
    print(ent.text, ent.start_char, ent.end_char, ent.label_,spacy.explain(ent.label_),  ent.sentiment)
print(matrix_tag)

0.3076923076923077
Harry Potter 0 12 PERSON People, including fictional 0.0
British 54 61 NORP Nationalities or religious or political groups 0.0
J. K. Rowling 69 82 PERSON People, including fictional 0.0
February 2018 6 19 DATE Absolute or relative dates or periods 0.0
more than 500 million 41 62 CARDINAL Numerals that do not fall under another type 0.0
Harry Potter 50 62 PERSON People, including fictional 0.0
Hermione Granger 80 96 PERSON People, including fictional 0.0
Ron Weasley 101 112 PERSON People, including fictional 0.0
Hogwarts School of Witchcraft and Wizardry 142 184 ORG Companies, agencies, institutions, etc. 0.0
four 9 13 CARDINAL Numerals that do not fall under another type 0.0
roughly eleven million 121 143 CARDINAL Numerals that do not fall under another type 0.0
the United States 154 171 GPE Countries, cities, states 0.0
twenty-four hours 179 196 TIME Times smaller than a day 0.0
['Harry Potter', 'PERSON(People, including fictional)', 'British', 'NORP(Nationalities o

### **Dependency Parse**

In [27]:
  print(candidate_ans)

['Harry Potter is a series of fantasy novels written by British author J. K. Rowling.', 'They have attracted a wide adult audience as well as younger readers and are often considered cornerstones of modern young adult literature.', 'As of February 2018, the books have sold more than 500 million copies worldwide, making them the best-selling book series in history, and have been translated into eighty languages.[3]', 'The novels chronicle the lives of a young wizard, Harry Potter, and his friends Hermione Granger and Ron Weasley, all of whom are students at Hogwarts School of Witchcraft and Wizardry.', 'The last four books consecutively set records as the fastest-selling books in history, with the final instalment selling roughly eleven million copies in the United States within twenty-four hours of its release.']


In [28]:
for token in en_doc:
    print("{0}/{1} <--{2}-- {3}/{4}".format(
        token.text, token.tag_, token.dep_, token.head.text, token.head.tag_))

Harry/NNP <--compound-- Potter/NNP
Potter/NNP <--nsubj-- is/VBZ
is/VBZ <--ROOT-- is/VBZ
a/DT <--det-- series/NN
series/NN <--attr-- is/VBZ
of/IN <--prep-- series/NN
fantasy/NN <--compound-- novels/NNS
novels/NNS <--pobj-- of/IN
written/VBN <--acl-- novels/NNS
by/IN <--agent-- written/VBN
British/JJ <--amod-- author/NN
author/NN <--compound-- Rowling/NNP
J./NNP <--compound-- Rowling/NNP
K./NNP <--compound-- Rowling/NNP
Rowling/NNP <--pobj-- by/IN
./. <--punct-- is/VBZ
The/DT <--det-- novels/NNS
novels/NNS <--nsubj-- chronicle/NN
chronicle/NN <--ROOT-- chronicle/NN
the/DT <--det-- lives/NNS
lives/NNS <--dobj-- chronicle/NN
of/IN <--prep-- lives/NNS
a/DT <--det-- wizard/NN
young/JJ <--amod-- wizard/NN
wizard/NN <--pobj-- of/IN
,/, <--punct-- wizard/NN
Harry/NNP <--compound-- Potter/NNP
Potter/NNP <--appos-- wizard/NN
,/, <--punct-- wizard/NN
and/CC <--cc-- wizard/NN
his/PRP$ <--poss-- friends/NNS
friends/NNS <--conj-- wizard/NN
Hermione/NNP <--compound-- Granger/NNP
Granger/NNP <--appos--

In [29]:
spacy.explain("NNP")

'noun, proper singular'

In [41]:
doc = nlp(input_text)

t = PrettyTable(['text', 'token.head.text','token.head.tag_','child','pos_','tag_','dep_','shape_',])
 
for token in doc:
  child =  ([child for child in token.children])
  t.add_row([token.text,token.head.text, spacy.explain(token.head.tag_), str(child), spacy.explain(token.pos_),spacy.explain(token.tag_) ,spacy.explain(token.dep_) ,
          spacy.explain(token.shape_)])
 
print(t)

+---------------+-----------------+-------------------------------------------+----------------------------------------------------+--------------------------+-------------------------------------------+----------------------------------------------+-----------------------------------+
|      text     | token.head.text |              token.head.tag_              |                       child                        |           pos_           |                    tag_                   |                     dep_                     |               shape_              |
+---------------+-----------------+-------------------------------------------+----------------------------------------------------+--------------------------+-------------------------------------------+----------------------------------------------+-----------------------------------+
|     Harry     |      Potter     |           noun, proper singular           |                         []                         |       

In [42]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Harry Potter 0 12 PERSON
British 54 61 NORP
J. K. Rowling 69 82 PERSON
Harry Potter 134 146 PERSON
Hermione Granger 164 180 PERSON
Ron Weasley 185 196 PERSON
Hogwarts School of Witchcraft and Wizardry 226 268 ORG
Harry 298 303 PERSON
Lord Voldemort 323 337 ORG
the Ministry of Magic 430 451 ORG
Muggles 482 489 ORG
first 537 542 ORDINAL
Harry Potter 550 562 PERSON
Philosopher 571 582 PERSON
26 June 1997 595 607 DATE
February 2018 848 861 DATE
more than 500 million 883 904 CARDINAL
four 1036 1040 CARDINAL
roughly eleven million 1148 1170 CARDINAL
the United States 1181 1198 GPE
twenty-four hours 1206 1223 TIME


In [43]:
root = [token for token in doc if token.head == token][0]
subject = list(root.lefts)[0]
for descendant in subject.subtree:
    assert subject is descendant or subject.is_ancestor(descendant)
    print(descendant.text, descendant.dep_, descendant.n_lefts,
            descendant.n_rights,
            [ancestor.text for ancestor in descendant.ancestors])

Harry compound 0 0 ['Potter', 'is']
Potter nsubj 1 0 ['is']


### **Sentiment**


In [44]:
 tricky_sentences = [
    "i really liked the movie and had fun. Sentiment",
    "VADER sentiment analysis is the shit.",
    "Sentiment analysis has never been good.",
    "Sentiment analysis with VADER has never been this good.",
    "Warren Beatty has never been so entertaining.",
    "I won't say that the movie is astounding and I wouldn't claim that \
    the movie is too banal either.",
    "I like to hate Michael Bay films, but I couldn't fault this one",
    "It's one thing to watch an Uwe Boll film, but another thing entirely \
    to pay for it",
    "The movie was too good",
    "This movie was actually neither that funny, nor super witty.",
    "This movie doesn't care about cleverness, wit or any other kind of \
    intelligent humor.",
    "Those who find ugly meanings in beautiful things are corrupt without \
    being charming.",
    "There are slow and repetitive parts, BUT it has just enough spice to \
    keep it interesting.",
    "The script is not fantastic, but the acting is decent and the cinematography \
    is EXCELLENT!",
    "Roger Dodger is one of the most compelling variations on this theme.",
    "Roger Dodger is one of the least compelling variations on this theme.",
    "Roger Dodger is at least compelling as a variation on the theme.",
    "they fall in love with the product",
    "but then it breaks",
    "usually around the time the 90 day warranty expires",
    "this is  excellent",
    "However, Mr. Carter solemnly argues, his client carried out the kidnapping \
    under orders and in the ''least offensive way possible.''"
 ]
 
sentiment_matrix =  []
sid = SentimentIntensityAnalyzer()
 for sentence in tricky_sentences :
     
     sent = sid.polarity_scores(sentence)
     if sent["compound"] > 0.05 :
       sentiment_matrix.append([sentence, "pos" ])
     elif  -0.5 < sent["compound"]  and sent["compound"] < 0.05 :
       sentiment_matrix.append([sentence, "neg" ])
     else:
        sentiment_matrix.append([sentence, "" ])

print(sentiment_matrix)      

[['i really liked the movie and had fun. Sentiment', 'pos'], ['VADER sentiment analysis is the shit.', 'pos'], ['Sentiment analysis has never been good.', 'neg'], ['Sentiment analysis with VADER has never been this good.', 'pos'], ['Warren Beatty has never been so entertaining.', 'pos'], ["I won't say that the movie is astounding and I wouldn't claim that    the movie is too banal either.", 'pos'], ["I like to hate Michael Bay films, but I couldn't fault this one", 'pos'], ["It's one thing to watch an Uwe Boll film, but another thing entirely    to pay for it", 'neg'], ['The movie was too good', 'pos'], ['This movie was actually neither that funny, nor super witty.', ''], ["This movie doesn't care about cleverness, wit or any other kind of    intelligent humor.", 'neg'], ['Those who find ugly meanings in beautiful things are corrupt without    being charming.', 'neg'], ['There are slow and repetitive parts, BUT it has just enough spice to    keep it interesting.', 'pos'], ['The script 

In [45]:
 ss = sid.polarity_scores("I AM GOOD")
 print(ss['compound'])

0.4404


### **Keras**

In [47]:
docs = ['Gut gemacht',
        'Gute arbeit',
        'Super idee',
        'Perfekt erledigt',
        'exzellent',
        'naja',
        'Schwache arbeit.',
        'Nicht gut',
        'Miese arbeit.',
        'Hätte es besser machen können.']
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)

[[9, 23], [34, 38], [13, 35], [21, 10], [7], [48], [19, 38], [21, 9], [36, 38], [33, 30, 6, 8, 42]]


In [51]:
from keras.preprocessing.text import Tokenizer

def vocab_creater(text_lists, VOCAB_SIZE):

  tokenizer = Tokenizer(num_words=VOCAB_SIZE)

  #fit_on_texts - Updates internal vocabulary based on a list of texts.   This method creates the vocabulary index based on word frequency. 
  #               So if you give it something like, "The cat sat on the mat." It will create a dictionary s.t.
  #               word_index["the"] = 1; word_index["cat"] = 2 it is word -> index dictionary so every word gets a unique integer value. 
  #               0 is reserved for padding. So lower integer means more frequent word (as they appear a lot).
  
  #texts_to_sequences - Transforms each text in texts to a sequence of integers.
  #                     So it basically takes each word in the text and replaces it with its corresponding integer value 
  #                     from the word_index dictionary. Nothing more, nothing less, certainly no magic involved.

  tokenizer.fit_on_texts(text_lists)
  dictionary = tokenizer.word_index
  
  word2idx = {}
  idx2word = {}
  for k, v in dictionary.items():
      if v < VOCAB_SIZE:
          word2idx[k] = v
          idx2word[v] = k
      if v >= VOCAB_SIZE-1:
          continue
          
  return word2idx, idx2word

# VOCAB_SIZE -> works on the frequescy of the word 
word2idx, idx2word = vocab_creater(["I am sourav Kumar and I live in India"], VOCAB_SIZE=14999)
print(word2idx)
print(idx2word)


{'i': 1, 'am': 2, 'sourav': 3, 'kumar': 4, 'and': 5, 'live': 6, 'in': 7, 'india': 8}
{1: 'i', 2: 'am', 3: 'sourav', 4: 'kumar', 5: 'and', 6: 'live', 7: 'in', 8: 'india'}


### **References**

*   https://spacy.io/universe/project/adam_qas
*   https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html

